In [11]:
from __future__ import division, print_function, absolute_import
import os
import time
import numpy as np
# np.object = object
# np.int = int
# np.bool = bool

import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

import tensorflow.keras

from autoencoder_helpers import makedirs, list_of_distancesc, list_of_distancese, print_and_write, list_of_norms
from data_preprocessing import batch_elastic_transform

# saver = tf.compat.v1.train.Saver()
from tensorflow.keras.utils import to_categorical
import sklearn.metrics as metrics
from sklearn.metrics import precision_score, recall_score

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

import sklearn.metrics as metrics
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import optuna

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

GPUID = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPUID)

tf.test.is_gpu_available()

True

In [12]:
import os
 
print("TensorFlow version is :",tf.__version__)
print("Keras version is :",tensorflow.keras.__version__)
print("GPU is","available" if tf.test.is_gpu_available()else "NOT AVAILABLE")
tf.config.list_physical_devices('GPU')

TensorFlow version is : 2.5.0
Keras version is : 2.5.0
GPU is available


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
from tensorflow.keras.layers import GaussianNoise

class_weights = {0: 1.0, 1: 8000 / 1000}

noise_factor = 0.1  # 根据实际情况调整噪声因子

train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda x: x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x.shape)
)

In [14]:
os.chdir('...\ISIC')

train_folder = '.../ISIC_2018/train'
val_folder = '.../ISIC_2018/test'
test_path = '.../ISIC_2018/test'

train_generator = train_datagen.flow_from_directory(train_folder,
                                                   target_size=(256, 256),
                                                   batch_size = 16,
                                                    color_mode='grayscale',
                                                   class_mode = 'binary',
                                                   classes = ['nmel', 'mel_aug'],
                                                     shuffle = True,
                                                   seed = 42)
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_folder,
                                                                       target_size=(256, 256),
                                                                       batch_size = 16,
                                                                       color_mode='grayscale',
                                                                       class_mode = 'binary',
                                                                       classes = ['nmel', 'mel'],
                                                                       seed = 42)

# print(len(val_generator))

Found 17806 images belonging to 2 classes.
Found 1512 images belonging to 2 classes.


In [ ]:
# def objective(trial):

learning_rate = 0.0002              # 0.0040906703512813
batch_size = 16
#     value_v1 = 30
#     value_v2 = 26
#     value_v3 = 18
#     value_v4 = 12
#     value_v5 = 20
value_v1 = 5
value_v2 = 4
value_v3 = 3
value_v4 = 2
value_v5 = 3
# value_v1 = 17
# value_v2 = 13
# value_v3 = 11
# value_v4 = 7
# value_v5 = 11

value_s1 = 3
value_s2 = 2
value_s3 = 2
value_s4 = 1
value_s5 = 1

# value_s1 = 4
# value_s2 = 1
# value_s3 = 1
# value_s4 = 4
# value_s5 = 5

n_map_1 = 36
n_map_2 = 40
# n_map_3 = 27
n_map_3 = 24
n_map_4 = 24
n_map_5 = 24

# lambda_class = 20
# lambda_2 = 10
# lambda_ae = 5            #0.25
# lambda_1 = 1

lambda_class = 100
lambda_2 = 19
lambda_ae = 1            #0.25
lambda_1 = 1

sigma = 6
alpha = 30

n_layers = 6

valid_ac_total = 0

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 2e-2)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    lambda_class = trial.suggest_int('lambda_class', 15, 25)
    lambda_ae = trial.suggest_int('lambda_ae', 1, 5)
    lambda_1 = trial.suggest_int('lambda_1', 1, 5)
    lambda_2 = trial.suggest_int('lambda_2', 1, 20)
    n_prototypes_n = trial.suggest_int('n_prototypes', 5, 6)
    n_prototypes = n_prototypes_n*2
    print('n_prototypes:',n_prototypes)
    sigma = trial.suggest_int('sigma', 1, 10)
    alpha_t = trial.suggest_int('alpha_t', 1, 6)
    alpha = alpha_t*10
    n_layers = trial.suggest_int('n_layers', 7, 8)
    n_map_1 = trial.suggest_int('n_map_1', 20, 40)
    n_map_2 = trial.suggest_int('n_map_2', 20, 40)
    n_map_3 = trial.suggest_int('n_map_3', 20, 40)
    n_map_4 = trial.suggest_int('n_map_4', 20, 40)
    n_map_5 = trial.suggest_int('n_map_5', 20, 40)
    value_s = trial.suggest_int('value_s', 1, 30)
    if value_v >= value_s:
    n_map_1 = trial.suggest_int('n_map_1', 20, 40)
    n_map_2 = trial.suggest_int('n_map_2', 20, 40)
    n_map_3 = trial.suggest_int('n_map_3', 20, 40)
    n_map_4 = trial.suggest_int('n_map_4', 20, 40)

accuracy_values1 = []
accuracy_values2 = []
precisiont = []
precisionv = []
recallt = []
recallv = []

n_prototypes = 10
#     n_layers = 8

best_accuracy = 0.0
best_precision = 0.0
best_recall = 0.0
best_learning_rate = None
best_batch_size = None

best_value_1 = None
best_value_2 = None

num = 0
fold_accuracies = []
fold_precisions = []
fold_recalls = []

# the directory to save the model
model_folder = os.path.join(os.getcwd(), "optuna", "5_layers")
makedirs(model_folder)
img_folder = os.path.join(model_folder, "img")
makedirs(img_folder)
model_filename = "cae_K-fold"
n_saves = None
console_log = open(os.path.join(model_folder, "console_log.txt"), "w+")

# training a
learning_rate = learning_rate                                                                               # 0.002
training_epochs = 200
#     batch_size = 34          # the size of a minibatch                                                  # 16
test_display_step = 50   # how many epochs we do evaluate on the test set once
save_step = 50          # how frequently do we save the model to disk

input_height = 256         # MNIST data input shape
input_width = input_height
n_input_channel = 1       # the number of color channels; for MNIST is 1.
input_size = input_height * input_width * n_input_channel   # the number of pixels in one input image
n_classes = 2

n_prototypes = n_prototypes         # the number of prototypes                                                  # 15
n_layers = n_layers                                                                                         # 4

f_1 = value_v1
f_2 = value_v2
f_3 = value_v3
f_4 = value_v4
f_5 = value_v5 # 3 3 3 3

s_1 = value_s1
s_2 = value_s2
s_3 = value_s3
s_4 = value_s4
s_5 = value_s5                                                                                             # 2 2 2 2

n_map_1 = n_map_1
n_map_2 = n_map_2
n_map_3 = n_map_3
n_map_4 = n_map_4                                                                                      # 32 32 32 10

filter_shape_1 = [f_1, f_1, n_input_channel, n_map_1]
filter_shape_2 = [f_2, f_2, n_map_1, n_map_2]
filter_shape_3 = [f_3, f_3, n_map_2, n_map_3]
filter_shape_4 = [f_4, f_4, n_map_3, n_map_4]
filter_shape_5 = [f_5, f_5, n_map_4, n_map_5]

stride_1 = [1, s_1, s_1, 1]
stride_2 = [1, s_2, s_2, 1]
stride_3 = [1, s_3, s_3, 1]
stride_4 = [1, s_4, s_4, 1]
stride_5 = [1, s_5, s_5, 1]

X = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, 65536], name='X')
X_img = tf.reshape(X, shape=[-1, input_height, input_width, n_input_channel], name='X_img')
Y = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, n_classes], name='Y')   # Y_shape==(None,2)
#     print(Y.shape)

lambda_class_t = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name="lambda_class_t")
lambda_ae_t = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name="lambda_ae_t")
lambda_2_t = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name="lambda_2_t")
lambda_1_t = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name="lambda_1_t")

#     weights = {
#         'enc_f1': tf.Variable(tf.random.normal(filter_shape_1,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='encoder_f1'),
#         'enc_f2': tf.Variable(tf.random.normal(filter_shape_2,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='encoder_f2'),
#         'enc_f3': tf.Variable(tf.random.normal(filter_shape_3,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='encoder_f3'),
#         'enc_f4': tf.Variable(tf.random.normal(filter_shape_4,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='encoder_f4'),
#         'enc_f5': tf.Variable(tf.random.normal(filter_shape_5,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='encoder_f5'),
#         'dec_f5': tf.Variable(tf.random.normal(filter_shape_5,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='decoder_f5'),
#         'dec_f4': tf.Variable(tf.random.normal(filter_shape_4,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='decoder_f4'),
#         'dec_f3': tf.Variable(tf.random.normal(filter_shape_3,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='decoder_f3'),
#         'dec_f2': tf.Variable(tf.random.normal(filter_shape_2,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='decoder_f2'),
#         'dec_f1': tf.Variable(tf.random.normal(filter_shape_1,
#                                                stddev=0.01,
#                                                dtype=tf.float32),
#                               name='decoder_f1')
#     }

weights = {
    'enc_f1': tf.Variable(tf.keras.initializers.he_normal()(filter_shape_1), name='encoder_f1'),
    'enc_f2': tf.Variable(tf.keras.initializers.he_normal()(filter_shape_2), name='encoder_f2'),
    'enc_f3': tf.Variable(tf.keras.initializers.he_normal()(filter_shape_3), name='encoder_f3'),
    'enc_f4': tf.Variable(tf.keras.initializers.he_normal()(filter_shape_4), name='encoder_f4'),
    'enc_fnew': tf.Variable(tf.keras.initializers.he_normal()(filter_shape_4), name='encoder_f4'),
    'enc_f5': tf.Variable(tf.keras.initializers.he_normal()(filter_shape_5), name='encoder_f5'),
    
    'dec_f5': tf.Variable(tf.random.normal(filter_shape_5,
                                           stddev=0.01,
                                           dtype=tf.float32),
                          name='decoder_f5'),
    'dec_fnew': tf.Variable(tf.random.normal(filter_shape_4,
                                           stddev=0.01,
                                           dtype=tf.float32),
                          name='decoder_f4'),
    'dec_f4': tf.Variable(tf.random.normal(filter_shape_4,
                                           stddev=0.01,
                                           dtype=tf.float32),
                          name='decoder_f4'),
    'dec_f3': tf.Variable(tf.random.normal(filter_shape_3,
                                           stddev=0.01,
                                           dtype=tf.float32),
                          name='decoder_f3'),
    'dec_f2': tf.Variable(tf.random.normal(filter_shape_2,
                                           stddev=0.01,
                                           dtype=tf.float32),
                          name='decoder_f2'),
    'dec_f1': tf.Variable(tf.random.normal(filter_shape_1,
                                           stddev=0.01,
                                           dtype=tf.float32),
                          name='decoder_f1')

}

biases = {
    'enc_b1': tf.Variable(tf.zeros([n_map_1], dtype=tf.float32),
                          name='encoder_b1'),
    'enc_b2': tf.Variable(tf.zeros([n_map_2], dtype=tf.float32),
                          name='encoder_b2'),
    'enc_b3': tf.Variable(tf.zeros([n_map_3], dtype=tf.float32),
                          name='encoder_b3'),
    'enc_b4': tf.Variable(tf.zeros([n_map_4], dtype=tf.float32),
                          name='encoder_b4'),
    'enc_bnew': tf.Variable(tf.zeros([n_map_4], dtype=tf.float32),
                          name='encoder_b4'),
    'enc_b5': tf.Variable(tf.zeros([n_map_5], dtype=tf.float32),
                          name='encoder_b5'),
    
    'dec_b5': tf.Variable(tf.zeros([n_map_4], dtype=tf.float32),
                          name='decoder_b5'),
    'dec_bnew': tf.Variable(tf.zeros([n_map_3], dtype=tf.float32),
                          name='decoder_b4'),
    'dec_b4': tf.Variable(tf.zeros([n_map_3], dtype=tf.float32),
                          name='decoder_b4'),
    'dec_b3': tf.Variable(tf.zeros([n_map_2], dtype=tf.float32),
                          name='decoder_b3'),
    'dec_b2': tf.Variable(tf.zeros([n_map_1], dtype=tf.float32),
                          name='decoder_b2'),
    'dec_b1': tf.Variable(tf.zeros([n_input_channel], dtype=tf.float32),
                          name='decoder_b1')
}

last_layer = {
    'w': tf.Variable(tf.random.uniform(shape=[n_prototypes, n_classes],
                                       dtype=tf.float32),
                     name='last_layer_w')
}

# padding can be either "SAME" or "VALID"
def conv_layer(input, filter, bias, strides, padding="VALID", nonlinearity=tf.nn.relu):
    conv = tf.nn.conv2d(input, filter, strides=strides, padding=padding)
    act = nonlinearity(conv + bias)
    return act

# tensorflow's conv2d_transpose needs to know the shape of the output
def deconv_layer(input, filter, bias, output_shape, strides, padding="VALID", nonlinearity=tf.nn.relu):
    deconv = tf.nn.conv2d_transpose(input, filter, output_shape, strides, padding=padding)
    act = nonlinearity(deconv + bias)
    return act

def fc_layer(input, weight, bias, nonlinearity=tf.nn.relu):
    return nonlinearity(tf.matmul(input, weight) + bias)

# construct the model
# eln means the output of the nth layer of the encoder
el1 = conv_layer(X_img, weights['enc_f1'], biases['enc_b1'], stride_1, "SAME")
el2 = conv_layer(el1, weights['enc_f2'], biases['enc_b2'], stride_2, "SAME")
el3 = conv_layer(el2, weights['enc_f3'], biases['enc_b3'], stride_3, "SAME")
el4 = conv_layer(el3, weights['enc_f4'], biases['enc_b4'], stride_4, "SAME")
elnew = conv_layer(el4, weights['enc_fnew'], biases['enc_bnew'], stride_4, "SAME")
el5 = conv_layer(elnew, weights['enc_f5'], biases['enc_b5'], stride_4, "SAME")

# we compute the output shape of each layer because the deconv_layer function requires it
l1_shape = el1.get_shape().as_list()
l2_shape = el2.get_shape().as_list()
l3_shape = el3.get_shape().as_list()
l4_shape = el4.get_shape().as_list()
lnew_shape = elnew.get_shape().as_list()
l5_shape = el5.get_shape().as_list()

flatten_size = l5_shape[1] * l5_shape[2] * l5_shape[3]
n_features = flatten_size
# feature vectors is the flattened output of the encoder
feature_vectors = tf.reshape(el5, shape=[-1, flatten_size], name='feature_vectors')
feature_vectors_dropout= tf.compat.v1.layers.dropout(inputs=feature_vectors, rate=0.3, training=False)

# the list prototype feature vectors
prototype_feature_vectors = tf.Variable(tf.random.uniform(shape=[n_prototypes, n_features],
                                                          dtype=tf.float32),
                                        name='prototype_feature_vectors')

deconv_batch_size = tf.identity(tf.shape(feature_vectors_dropout)[0], name="deconv_batch_size")

# this is necessary for prototype images evaluation
reshape_feature_vectors = tf.reshape(feature_vectors_dropout, shape=[-1, l5_shape[1], l5_shape[2], l5_shape[3]])

# dln means the output of the nth layer of the decoder
dl5 = deconv_layer(reshape_feature_vectors, weights['dec_f5'], biases['dec_b5'],
                   output_shape=[deconv_batch_size, l4_shape[1], l4_shape[2], l4_shape[3]],
                   strides=stride_5, padding="SAME")
dlnew = deconv_layer(dl5, weights['dec_f4'], biases['dec_b4'],
               output_shape=[deconv_batch_size, l3_shape[1], l3_shape[2], l3_shape[3]],
               strides=stride_4, padding="SAME")
dl4 = deconv_layer(dlnew, weights['dec_f4'], biases['dec_b4'],
               output_shape=[deconv_batch_size, l3_shape[1], l3_shape[2], l3_shape[3]],
               strides=stride_4, padding="SAME")
dl3 = deconv_layer(dl4, weights['dec_f3'], biases['dec_b3'],
                   output_shape=[deconv_batch_size, l2_shape[1], l2_shape[2], l2_shape[3]],
                   strides=stride_3, padding="SAME")
dl2 = deconv_layer(dl3, weights['dec_f2'], biases['dec_b2'],
                   output_shape=[deconv_batch_size, l1_shape[1], l1_shape[2], l1_shape[3]],
                   strides=stride_2, padding="SAME")
dl1 = deconv_layer(dl2, weights['dec_f1'], biases['dec_b1'],
                   output_shape=[deconv_batch_size, input_height, input_width, n_input_channel],
                   strides=stride_1, padding="SAME", nonlinearity=tf.nn.sigmoid)

X_decoded = tf.reshape(dl1, shape=[-1, input_size], name='X_decoded')
X_true = tf.identity(X, name='X_true')

prototype_distances = list_of_distancesc(feature_vectors_dropout,
                                        prototype_feature_vectors)
#     prototype_distances = list_of_distancese(feature_vectors_dropout,
#                                             prototype_feature_vectors)
prototype_distances = tf.identity(prototype_distances, name='prototype_distances')
feature_vector_distances = list_of_distancese(prototype_feature_vectors,
                                             feature_vectors_dropout)
#     feature_vector_distances = list_of_distancesc(prototype_feature_vectors,
#                                                  feature_vectors_dropout)
feature_vector_distances = tf.identity(feature_vector_distances, name='feature_vector_distances')

#     print(tf.shape(prototype_distances))
#     print(tf.shape(feature_vector_distances))

#     print(prototype_distances.shape)
#     print(last_layer['w'].shape)

logits = tf.matmul(prototype_distances, last_layer['w'], name='logits')
probability_distribution = tf.nn.softmax(logits=logits,
                                         name='probability_distribution')

ae_error = tf.reduce_mean(list_of_norms(X_decoded - X_true), name='ae_error')
class_error = tf.keras.losses.BinaryCrossentropy(from_logits=True)(Y, logits)
class_error = tf.identity(class_error, name='class_error')
error_1 = tf.reduce_mean(tf.reduce_max(feature_vector_distances, axis = 1), name='error_1')
error_2 = tf.reduce_mean(tf.reduce_max(prototype_distances, axis = 1), name='error_2')

total_error = lambda_class_t * class_error +\
              lambda_ae_t * ae_error + \
              lambda_1_t * error_1 + \
              lambda_2_t * error_2
total_error = tf.identity(total_error, name='total_error')

correct_prediction = tf.equal(tf.argmax(logits, 1),
                              tf.argmax(Y, 1),
                              name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32),
                          name='accuracy')

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(total_error)
#     optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).maximize(accuracy)
tf.compat.v1.add_to_collection("optimizer", optimizer)

init = tf.compat.v1.global_variables_initializer()

hyperparameters = {
    "learning_rate": learning_rate,
    "training_epochs": training_epochs,
    "batch_size": batch_size,
    "test_display_step": test_display_step,
    "save_step": save_step,

    "lambda_class": lambda_class,
    "lambda_ae": lambda_ae,
    "lambda_1": lambda_1,
    "lambda_2": lambda_2,

    "input_height": input_height,
    "input_width": input_width,
    "n_input_channel": n_input_channel,
    "input_size": input_size,
    "n_classes": n_classes,

    "n_prototypes": n_prototypes,
    "n_layers": n_layers,

    "f_1":	f_1,
    "f_2":	f_2,
    "f_3": 	f_3,
    "f_4": 	f_4,

    "s_1" :s_1,
    "s_2": s_2,
    "s_3": s_3,
    "s_4": s_4,

    "n_map_1": n_map_1,
    "n_map_2": n_map_2,
    "n_map_3": n_map_3,
    "n_map_4": n_map_4,

    "n_features": n_features
}
for (name, value) in hyperparameters.items():
    tf.compat.v1.add_to_collection('hyperparameters', tf.constant(name=name, value=value))

#     print(type(feature_vector_distances))
saver = tf.compat.v1.train.Saver(max_to_keep=n_saves)

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7

total_distance = 0.0
num_pairs = 0

total_distance_encoder_decoder = 0.0

with tf.compat.v1.Session(config=config) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(init)
    num = 0
    n_train_batch = len(train_generator) // batch_size 
    n_valid_batch = len(val_generator) // batch_size
#         print('len(val_generator)',len(val_generator))
#         print('n_valid_batch',n_valid_batch)
    for epoch in range(training_epochs):
        print_and_write("#"*80, console_log)
        print_and_write("Epoch: %04d" % (epoch), console_log)
        start_time = time.time()

        train_ce, train_ae, train_e1, train_e2, train_te, train_ac = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        train_precision = 0.0
        train_loss = 0.0
        train_recall = 0.0

        svalid_ce, svalid_ae, svalid_e1, svalid_e2, svalid_te, svalid_ac = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        svalid_precision = 0.0
        svalid_loss = 0.0
        svalid_recall = 0.0

        total_distance = 0.0

        y_true_t = []  # 真实标签
        y_pred_t = []  # 预测结果
        y_true_v = []  # 真实标签
        y_pred_v = []  # 预测结果
        sy_true_v = []  # 真实标签
        sy_pred_v = []  # 预测结果

        # Loop over all batches
        num = 0
        for i in range(n_train_batch):
            batch_x, batch_y = next(train_generator)
            batch_x = batch_x.reshape(-1, 65536)
            batch_y = to_categorical(batch_y, num_classes=n_classes)

            oversampler = RandomOverSampler(sampling_strategy='auto')  # Use 'auto' to automatically balance classes
            batch_x_resampled, batch_y_resampled = oversampler.fit_resample(batch_x, batch_y)

            batch_y_resampled = to_categorical(batch_y_resampled, num_classes=n_classes)

            _t, cet, aet, e1t, e2t, tet, act = sess.run(
                (optimizer,
                 class_error,
                 ae_error,
                 error_1,
                 error_2,
                 total_error,
                 accuracy),
                feed_dict={X: batch_x_resampled,
                           Y: batch_y_resampled, 
                           lambda_class_t: lambda_class,
                           lambda_ae_t: lambda_ae,
                           lambda_1_t: lambda_1,
                           lambda_2_t: lambda_2})
            train_ce += (cet/n_train_batch)
            train_ae += (aet/n_train_batch)
            train_e1 += (e1t/n_train_batch)
            train_e2 += (e2t/n_train_batch)
            train_te += (tet/n_train_batch)
            train_ac += (act/n_train_batch)
            end_time = time.time()

            predictions_t = sess.run(probability_distribution, feed_dict={X: batch_x_resampled})
            batch_pred_t = np.argmax(predictions_t, axis=1)
            y_pred_t.extend(batch_pred_t)
            y_true_t.extend(np.argmax(batch_y_resampled, axis=1))

        print_and_write("training set errors:", console_log)
        print_and_write("\tclassification error: {:.6f}".format(train_ce), console_log)
        print_and_write("\tautoencoder error: {:.6f}".format(train_ae), console_log)
        print_and_write("\terror_1: {:.6f}".format(train_e1), console_log)
        print_and_write("\terror_2: {:.6f}".format(train_e2), console_log)
        print_and_write("\ttotal error: {:.6f}".format(train_te), console_log)
        print_and_write("\taccuracy: {:.4f}".format(train_ac), console_log)
        print_and_write('training takes {0:.2f} seconds.'.format((end_time - start_time)), console_log)

        precision_t = precision_score(y_true_t, y_pred_t)
        recall_t = recall_score(y_true_t, y_pred_t)

        print('recall_t',recall_t)
        print('precision_t',precision_t)


        valid_ce, valid_ae, valid_e1, valid_e2, valid_te, valid_ac = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
        valid_precision = 0.0
        valid_loss = 0.0
        valid_recall = 0.0

        for j in range(n_valid_batch):
            batchv_x, batchv_y = next(val_generator)
            batchv_y = to_categorical(batchv_y, num_classes=n_classes)
            batchv_x = batchv_x.reshape(-1, 65536)

            ce, ae, e1, e2, te, ac = sess.run(
                (class_error,
                 ae_error,
                 error_1,
                 error_2,
                 total_error,
                 accuracy),
                feed_dict={X: batchv_x,
                           Y: batchv_y,
                           lambda_class_t: lambda_class,
                           lambda_ae_t: lambda_ae,
                           lambda_2_t: lambda_2,
                           lambda_1_t: lambda_1})
            valid_ce += ce / n_valid_batch
            valid_ae += ae / n_valid_batch
            valid_e1 += e1 / n_valid_batch
            valid_e2 += e2 / n_valid_batch
            valid_te += te / n_valid_batch
            valid_ac += ac / n_valid_batch
            predictions_v = sess.run(probability_distribution, feed_dict={X: batchv_x})
            batch_pred_v = np.argmax(predictions_v, axis=1)
            y_pred_v.extend(batch_pred_v)
            y_true_v.extend(np.argmax(batchv_y, axis=1))

        print('valid-set:')
        print_and_write("\tvalidation set errors:", console_log)
        print_and_write("\tclassification error: {:.6f}".format(valid_ce), console_log)
        print_and_write("\tautoencoder error: {:.6f}".format(valid_ae), console_log)
        print_and_write("\terror_1: {:.6f}".format(valid_e1), console_log)
        print_and_write("\terror_2: {:.6f}".format(valid_e2), console_log)
        print_and_write("\ttotal error: {:.6f}".format(valid_te), console_log)
        print_and_write("\taccuracy: {:.4f}".format(valid_ac), console_log)

        precision_v = precision_score(y_true_v, y_pred_v)
        recall_v = recall_score(y_true_v, y_pred_v)

        print('recall_v',recall_v)
        print('precision_v',precision_v)

#             if epoch % 10 == 0 or epoch == training_epochs - 1:       # save_step == 50 --> 每50个epoch保存一次

        saver.save(sess, os.path.join(model_folder, model_filename + str(epoch)),
                   global_step=epoch,
                   write_meta_graph=(epoch == 0 or epoch == training_epochs - 1))

        prototype_imgs = sess.run(X_decoded,
                                  feed_dict={feature_vectors_dropout: prototype_feature_vectors.eval()})

        n_cols = int(n_prototypes/2)
        n_rows = n_prototypes // n_cols + 1 if n_prototypes % n_cols != 0 else n_prototypes // n_cols
        g, b = plt.subplots(n_rows, n_cols, figsize=(n_cols, n_rows))
        for i in range(n_rows):
            for j in range(n_cols):
                if i * n_cols + j < n_prototypes:
#                             b[i][j].imshow(prototype_imgs[i * n_cols + j].reshape(input_height, input_width),
#                                            cmap='gray')
                    b[i][j].imshow(prototype_imgs[i * n_cols + j].reshape(input_height, input_width, n_input_channel),
                                   interpolation='none')
                    b[i][j].axis('off')

                    if num_pairs == 0:
#                                 distance = np.linalg.norm(prototype_imgs[i * n_cols + j] - prototype_imgs[num_pairs])
#                                 total_distance = 0
                        num_pairs += 1
                    else:
                        distance = np.linalg.norm(prototype_imgs[i * n_cols + j] - prototype_imgs[i * n_cols + j - 1])
                        total_distance += distance
                        num_pairs += 1

        plt.savefig(os.path.join(img_folder, f'prototype_{str(lambda_class)}_{str(lambda_ae)}_{str(lambda_1)}_{str(lambda_2)}_{str(epoch)}.png'),
                    transparent=True,
                    bbox_inches='tight',
                    pad_inches=0)
        plt.close()

#                 if epoch % 10 == 0 or epoch == training_epochs - 1:

        encode_decode = sess.run(X_decoded,
#                                      feed_dict={X: next(train_generator)[:examples_to_show]})
#                                          feed_dict={X: batch_x_resampled})
                            feed_dict={X: batch_x_resampled})
        # Compare original images with their reconstructions
        f, a = plt.subplots(2, 10, figsize=(10, 2))
        for i in range(10):
#                         a[0][i].imshow(batch_x_resampled[i].reshape(input_height, input_width),
            a[0][i].imshow(batch_x_resampled[i].reshape(input_height, input_width),
                            cmap='gray',
                            interpolation='none')
            a[0][i].axis('off')
            a[1][i].imshow(encode_decode[i].reshape(input_height, input_width), 
                            cmap='gray',
                            interpolation='none')
            a[1][i].axis('off')

#                         distance_encoder_decoder = np.linalg.norm(batch_x_resampled[i] - encode_decode[i])
            distance_encoder_decoder = np.linalg.norm(batch_x_resampled[i] - encode_decode[i])

            total_distance_encoder_decoder += distance_encoder_decoder

        plt.savefig(os.path.join(img_folder, f'decoding_{str(lambda_class)}_{str(lambda_ae)}_{str(lambda_1)}_{str(lambda_2)}_{str(epoch)}.png'),
                    transparent=True,
                    bbox_inches='tight',
                    pad_inches=0)
        plt.close()

#                     print('n_layers',n_layers)

        print('total_distance',total_distance)
        print('total_distance_encoder_decoder',total_distance_encoder_decoder)

        valid_ac_total += valid_ac
        av_ac = valid_ac_total/(epoch+1)
        print('valid_ac_av',av_ac)
#     return valid_ac_total 

################################################################################
Epoch: 0000
training set errors:
	classification error: 2.749083
	autoencoder error: 1928.160403
	error_1: 2155.384693
	error_2: 1.053227
	total error: 4378.464695
	accuracy: 0.5000
training takes 14.91 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 2.770307
	autoencoder error: 1317.481812
	error_1: 1680.093359
	error_2: 1.024456
	total error: 3294.070459
	accuracy: 0.0750
recall_v 1.0
precision_v 0.075
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold0-0 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 8.077014088630676
total_distance_encoder_decoder 352.5914192199707
valid_ac_av 0.075
################################################################################
Epoch: 0001
training set errors:
	classification error: 2.534448
	autoencoder error: 1354.509394
	error_1: 1384.444244
	error_2: 0.982747
	total error: 3011.070673
	a

total_distance 282.6910285949707
total_distance_encoder_decoder 2756.5025930404663
valid_ac_av 0.10795454545454546
################################################################################
Epoch: 0011
training set errors:
	classification error: 1.222898
	autoencoder error: 310.709244
	error_1: 456.384342
	error_2: 0.571667
	total error: 900.245044
	accuracy: 0.5000
training takes 10.49 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 1.261210
	autoencoder error: 264.120181
	error_1: 437.857410
	error_2: 0.564559
	total error: 838.825256
	accuracy: 0.1125
recall_v 1.0
precision_v 0.1125
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold11-11 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 295.75868797302246
total_distance_encoder_decoder 2955.2127804756165
valid_ac_av 0.10833333333333334
################################################################################
Epoch: 0012
training set errors:
	clas

total_distance 289.0620651245117
total_distance_encoder_decoder 4180.247360706329
valid_ac_av 0.11420454545454546
################################################################################
Epoch: 0022
training set errors:
	classification error: 0.919660
	autoencoder error: 137.304445
	error_1: 172.015429
	error_2: 0.466448
	total error: 410.148435
	accuracy: 0.5000
training takes 8.20 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.964485
	autoencoder error: 149.370580
	error_1: 166.617523
	error_2: 0.469246
	total error: 421.352240
	accuracy: 0.1125
recall_v 1.0
precision_v 0.1125
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold22-22 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 279.64151191711426
total_distance_encoder_decoder 4313.001779079437
valid_ac_av 0.11413043478260869
################################################################################
Epoch: 0023
training set errors:
	classif

total_distance 209.46636962890625
total_distance_encoder_decoder 5360.53585767746
valid_ac_av 0.11136363636363636
################################################################################
Epoch: 0033
training set errors:
	classification error: 0.789478
	autoencoder error: 106.054593
	error_1: 64.426115
	error_2: 0.404727
	total error: 257.118295
	accuracy: 0.5000
training takes 8.26 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.820895
	autoencoder error: 125.526172
	error_1: 60.905097
	error_2: 0.401790
	total error: 276.154773
	accuracy: 0.1250
recall_v 1.0
precision_v 0.125
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold33-33 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 205.44465446472168
total_distance_encoder_decoder 5461.490804672241
valid_ac_av 0.11176470588235293
################################################################################
Epoch: 0034
training set errors:
	classifica

total_distance 158.56653594970703
total_distance_encoder_decoder 6402.823826313019
valid_ac_av 0.11448863636363636
################################################################################
Epoch: 0044
training set errors:
	classification error: 0.732028
	autoencoder error: 97.310628
	error_1: 28.732946
	error_2: 0.357902
	total error: 206.046484
	accuracy: 0.5000
training takes 8.13 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.758926
	autoencoder error: 82.390746
	error_1: 23.384034
	error_2: 0.350255
	total error: 188.322198
	accuracy: 0.1000
recall_v 1.0
precision_v 0.1
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold44-44 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 154.4095401763916
total_distance_encoder_decoder 6487.750986337662
valid_ac_av 0.11416666666666665
################################################################################
Epoch: 0045
training set errors:
	classification

...

total_distance 100.11019706726074
total_distance_encoder_decoder 8237.587096214294
valid_ac_av 0.11174242424242423
################################################################################
Epoch: 0066
training set errors:
	classification error: 0.697608
	autoencoder error: 86.851066
	error_1: 13.488745
	error_2: 0.274195
	total error: 175.310288
	accuracy: 0.5000
training takes 8.21 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.715090
	autoencoder error: 106.705093
	error_1: 15.540596
	error_2: 0.271027
	total error: 198.904196
	accuracy: 0.1250
recall_v 1.0
precision_v 0.125
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold66-66 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 99.10683727264404
total_distance_encoder_decoder 8332.855021476746
valid_ac_av 0.11194029850746268
################################################################################
Epoch: 0067
training set errors:
	classificat

total_distance 80.55083990097046
total_distance_encoder_decoder 9164.700137138367
valid_ac_av 0.11266233766233764
################################################################################
Epoch: 0077
training set errors:
	classification error: 0.693633
	autoencoder error: 74.642184
	error_1: 9.681634
	error_2: 0.234901
	total error: 158.150227
	accuracy: 0.5000
training takes 8.18 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.708095
	autoencoder error: 90.543831
	error_1: 11.127031
	error_2: 0.235601
	total error: 176.956772
	accuracy: 0.1125
recall_v 1.0
precision_v 0.1125
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold77-77 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 77.52615404129028
total_distance_encoder_decoder 9270.298498153687
valid_ac_av 0.11266025641025641
################################################################################
Epoch: 0078
training set errors:
	classificatio

total_distance 61.70035791397095
total_distance_encoder_decoder 10089.238080978394
valid_ac_av 0.11264204545454543
################################################################################
Epoch: 0088
training set errors:
	classification error: 0.692584
	autoencoder error: 77.848996
	error_1: 9.390910
	error_2: 0.199992
	total error: 160.298174
	accuracy: 0.5000
training takes 8.13 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.707374
	autoencoder error: 89.972432
	error_1: 11.604013
	error_2: 0.201661
	total error: 176.145393
	accuracy: 0.1000
recall_v 1.0
precision_v 0.1
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold88-88 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 58.888978004455566
total_distance_encoder_decoder 10178.930206775665
valid_ac_av 0.11249999999999998
################################################################################
Epoch: 0089
training set errors:
	classificatio

total_distance 44.770620584487915
total_distance_encoder_decoder 10980.971287488937
valid_ac_av 0.11287878787878786
################################################################################
Epoch: 0099
training set errors:
	classification error: 0.691785
	autoencoder error: 68.972730
	error_1: 8.330265
	error_2: 0.166116
	total error: 149.637745
	accuracy: 0.5000
training takes 8.13 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.701444
	autoencoder error: 67.275185
	error_1: 8.223086
	error_2: 0.162515
	total error: 148.730420
	accuracy: 0.1875
recall_v 1.0
precision_v 0.1875
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold99-99 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 44.54519748687744
total_distance_encoder_decoder 11060.053675413132
valid_ac_av 0.11362499999999998
################################################################################
Epoch: 0100
training set errors:
	classificat

total_distance 33.871246337890625
total_distance_encoder_decoder 11839.090796470642
valid_ac_av 0.11318181818181813
################################################################################
Epoch: 0110
training set errors:
	classification error: 0.691741
	autoencoder error: 72.492272
	error_1: 7.631578
	error_2: 0.138485
	total error: 151.929181
	accuracy: 0.5000
training takes 8.18 seconds.
recall_t 1.0
precision_t 0.5
valid-set:
	validation set errors:
	classification error: 0.705352
	autoencoder error: 160.184212
	error_1: 9.946953
	error_2: 0.135318
	total error: 243.237369
	accuracy: 0.1750
recall_v 1.0
precision_v 0.175
INFO:tensorflow:F:\ZYR\ISIC\optuna\5_layers\cae_K-fold110-110 is not in all_model_checkpoint_paths. Manually adding it.
total_distance 33.379183530807495
total_distance_encoder_decoder 11949.975468635559
valid_ac_av 0.11373873873873869
################################################################################
Epoch: 0111


In [ ]:
study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=10) 

best_params = study.best_params
best_value = study.best_value
print(f"Best Parameters: {best_params}")
print(f"Best Value: {best_value}")

